<a href="https://colab.research.google.com/github/mythrikulkarni/Aggie-Assist/blob/main/AggieAssist1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

In [2]:
!pip install simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import json
with open(r"/train.json", "r") as read_file:
    train = json.load(read_file)

In [11]:
train

[{'context': "There are many types of mental health professionals. The variety of providers and their services may be confusing. Each have various levels of education, training, and may have different areas of expertise. Finding the professional who best fits your needs may require some research. To find resources on campus for mental health, here's a helpful resource: https://shcs.ucdavis.edu/services/counseling-services",
  'qas': [{'id': '00001',
    'is_impossible': False,
    'question': 'What is the difference between mental health professionals?',
    'answers': [{'text': "There are many types of mental health professionals. The variety of providers and their services may be confusing. Each have various levels of education, training, and may have different areas of expertise. Finding the professional who best fits your needs may require some research. To find resources on campus for mental health, here's a helpful resource: https://shcs.ucdavis.edu/services/counseling-services",

In [12]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [13]:
model_type="bert"
model_name= "bert-base-cased"
if model_type == "bert":
    model_name = "bert-base-cased"

elif model_type == "roberta":
    model_name = "roberta-base"

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"

In [22]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 15
model_args.evaluate_during_training = True
model_args.n_best_size=1
model_args.num_train_epochs=100

In [23]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 100,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3, 
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [24]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args, use_cuda= False
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and a

In [25]:
!rm -rf outputs


In [29]:
with open(r"/predictions.json", "r") as read_file:
    to_predict = json.load(read_file)

In [30]:
answers, probabilities = model.predict(to_predict)

print(answers)

add example index and unique id: 100%|██████████| 15/15 [00:00<00:00, 81920.00it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[{'id': '00016', 'answer': ["Anyone can post anything online. It's up to the readers to think critically about information to decide if it seems helpful, truthful, or safe."]}, {'id': '00017', 'answer': ['anxiety, boosting motivation, and improving decision-making by giving you a clear roadmap to follow towards achieving your goals.', 'you a clear roadmap to follow towards achieving your goals.', 'your mental health by reducing anxiety, boosting motivation, and improving decision-making by giving you a clear roadmap to follow towards achieving your goals.']}, {'id': '00018', 'answer': ['empty']}, {'id': '00019', 'answer': ['empty']}, {'id': '00020', 'answer': ['empty']}, {'id': '00021', 'answer': ['their family members and friends. Some support groups are peer-led, while others may be led by a mental health professional.']}, {'id': '00022', 'answer': ['/emergency/patients/index.html), which is located at 2315 Stockton Blvd, Sacramento, CA 95817. You can also call 911 for emergency serv